In [ ]:
import numpy as np
import pandas as pd
import os

print("Exploring data directory structure...")
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Install ultralytics (YOLO)
!pip install -U ultralytics

# Verify installation
import ultralytics
ultralytics.checks()

# Import required libraries
import json
import os
from sklearn.model_selection import train_test_split
import shutil
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

In [ ]:
# Load and examine the TACO annotations
json_path = './data/annotations.json'

with open(json_path, 'r') as f:
    data = json.load(f)

images = data['images']
annotations = data['annotations']
categories = data['categories']

print(f"📊 TACO Dataset Overview:")
print(f"   Total images: {len(images)}")
print(f"   Total annotations: {len(annotations)}")
print(f"   Total categories: {len(categories)}")
print(f"   Average annotations per image: {len(annotations)/len(images):.2f}")

# Show first few categories
print(f"\n🏷️  First 10 categories:")
for i, cat in enumerate(categories[:10]):
    print(f"   {cat['id']}: {cat['name']}")

# Show dataset distribution
print(f"\n📁 Image file structure (first 5):")
for img in images[:5]:
    print(f"   {img['file_name']} ({img['width']}x{img['height']})")

In [ ]:
def convert_taco_to_yolo(json_path, image_root_dir, output_dir):
    """
    Convert TACO dataset from COCO format to YOLO format

    Args:
        json_path: Path to annotations.json
        image_root_dir: Root directory containing batch folders with images
        output_dir: Output directory for YOLO format dataset
    """

    print("🔄 Loading TACO annotations...")
    with open(json_path, 'r') as f:
        data = json.load(f)

    images = data['images']
    annotations = data['annotations']
    categories = data['categories']

    print(f"Found {len(images)} images, {len(annotations)} annotations, {len(categories)} categories")

    # Create output directories
    print("📁 Creating output directories...")
    os.makedirs(os.path.join(output_dir, 'images/train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images/val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/val'), exist_ok=True)

    # Split dataset into train/val
    image_ids = [img['id'] for img in images]
    train_ids, val_ids = train_test_split(image_ids, test_size=0.2, random_state=42)

    print(f"📊 Train set: {len(train_ids)} images")
    print(f"📊 Validation set: {len(val_ids)} images")

    # Create lookup dictionaries
    image_id_to_filename = {img['id']: img['file_name'] for img in images}
    category_id_to_name = {cat['id']: cat['name'] for cat in categories}

    # Create mapping from category_id to YOLO class_id (0-based indexing)
    unique_category_ids = sorted(list(set(cat['id'] for cat in categories)))
    category_id_to_yolo_id = {cat_id: idx for idx, cat_id in enumerate(unique_category_ids)}

    print("🔄 Converting images and annotations...")
    copied_count = 0
    missing_count = 0

    for i, img in enumerate(images):
        if i % 500 == 0:
            print(f"   Processed {i}/{len(images)} images...")

        img_id = img['id']
        filename = img['file_name']  # This should be like "batch_1/000001.jpg"
        img_width = img['width']
        img_height = img['height']

        # Determine train or val
        if img_id in train_ids:
            image_dir = os.path.join(output_dir, 'images/train')
            label_dir = os.path.join(output_dir, 'labels/train')
        else:
            image_dir = os.path.join(output_dir, 'images/val')
            label_dir = os.path.join(output_dir, 'labels/val')

        # Construct full image path
        full_image_path = os.path.join(image_root_dir, filename)

        # Extract just the image filename for destination
        image_filename = os.path.basename(filename)

        if os.path.exists(full_image_path):
            # Copy image
            shutil.copy(full_image_path, os.path.join(image_dir, image_filename))
            copied_count += 1
        else:
            if missing_count < 10:  # Only print first 10 missing files
                print(f"⚠️  Warning: Image {full_image_path} not found.")
            missing_count += 1
            continue

        # Create label file
        label_filename = image_filename.replace('.jpg', '.txt').replace('.JPG', '.txt')
        label_file = os.path.join(label_dir, label_filename)

        with open(label_file, 'w') as lf:
            for ann in annotations:
                if ann['image_id'] == img_id:
                    category_id = ann['category_id']
                    yolo_class_id = category_id_to_yolo_id[category_id]

                    bbox = ann['bbox']  # [x, y, width, height] in COCO format

                    # Convert to YOLO format (normalized center coordinates)
                    x_center = (bbox[0] + bbox[2] / 2) / img_width
                    y_center = (bbox[1] + bbox[3] / 2) / img_height
                    width = bbox[2] / img_width
                    height = bbox[3] / img_height

                    lf.write(f"{yolo_class_id} {x_center} {y_center} {width} {height}\n")

    print(f"✅ Successfully copied {copied_count} images")
    print(f"❌ Missing images: {missing_count}")

    return category_id_to_yolo_id, category_id_to_name

def create_data_yaml(output_dir, categories, category_id_to_yolo_id):
    """Create data.yaml file for YOLO training"""

    # Create class names list in YOLO order (0-based indexing)
    class_names = [""] * len(category_id_to_yolo_id)
    for cat in categories:
        yolo_id = category_id_to_yolo_id[cat['id']]
        class_names[yolo_id] = cat['name']

    yaml_content = f"""# TACO Dataset YOLO Configuration
train: {os.path.abspath(os.path.join(output_dir, 'images/train'))}
val: {os.path.abspath(os.path.join(output_dir, 'images/val'))}

nc: {len(class_names)}
names: {class_names}
"""

    yaml_path = os.path.join(output_dir, 'data.yaml')
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)

    print(f"📄 Created {yaml_path}")
    print(f"📊 Number of classes: {len(class_names)}")
    print("🏷️  Class names:", class_names[:10], "..." if len(class_names) > 10 else "")

    return yaml_path

print("✅ Conversion functions defined!")

In [ ]:
json_path = './data/annotations.json'
image_root_dir = './data'  # Directory containing batch_1, batch_2, etc.
output_dir = './output'

print("🚀 Starting TACO to YOLO conversion...")
print(f"📁 JSON path: {json_path}")
print(f"📁 Image root: {image_root_dir}")
print(f"📁 Output directory: {output_dir}")

# Convert dataset
category_id_to_yolo_id, category_id_to_name = convert_taco_to_yolo(
    json_path, image_root_dir, output_dir
)

# Load categories for YAML creation
with open(json_path, 'r') as f:
    data = json.load(f)
categories = data['categories']

# Create data.yaml
yaml_path = create_data_yaml(output_dir, categories, category_id_to_yolo_id)

print("\n" + "="*50)
print("🎉 Conversion complete!")
print(f"📁 YOLO dataset created in: {output_dir}")
print(f"📄 Configuration file: {yaml_path}")
print("="*50)

In [ ]:
# Display sample images with annotations
import matplotlib.pyplot as plt
import cv2
import random

def visualize_yolo_sample(output_dir, num_samples=4):
    """Visualize sample images with YOLO annotations"""

    # Load class names
    yaml_path = os.path.join(output_dir, 'data.yaml')
    with open(yaml_path, 'r') as f:
        yaml_content = f.read()

    # Extract class names (simple parsing)
    names_line = [line for line in yaml_content.split('\n') if line.startswith('names:')][0]
    class_names = eval(names_line.split('names: ')[1])

    # Get sample training images
    train_img_dir = os.path.join(output_dir, 'images/train')
    train_label_dir = os.path.join(output_dir, 'labels/train')

    image_files = [f for f in os.listdir(train_img_dir) if f.endswith('.jpg')]
    sample_files = random.sample(image_files, min(num_samples, len(image_files)))

    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()

    for i, img_file in enumerate(sample_files):
        # Load image
        img_path = os.path.join(train_img_dir, img_file)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load annotations
        label_file = img_file.replace('.jpg', '.txt')
        label_path = os.path.join(train_label_dir, label_file)

        h, w = image.shape[:2]

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                annotations = f.readlines()

            # Draw bounding boxes
            for ann in annotations:
                class_id, x_center, y_center, width, height = map(float, ann.strip().split())

                # Convert from YOLO to pixel coordinates
                x1 = int((x_center - width/2) * w)
                y1 = int((y_center - height/2) * h)
                x2 = int((x_center + width/2) * w)
                y2 = int((y_center + height/2) * h)

                # Draw rectangle
                cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

                # Add label
                label = class_names[int(class_id)]
                cv2.putText(image, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        axes[i].imshow(image)
        axes[i].set_title(f'Sample {i+1}: {img_file}')
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

print("📊 Visualizing sample training data...")
visualize_yolo_sample('./output')

In [ ]:
print("🚀 Starting YOLO training...")

# Initialize YOLOv8 model with pretrained weights
#model = YOLO('yolov8n.pt')  # nano model (fastest)
# Alternative options:
# model = YOLO('yolov8s.pt')  # small model (better accuracy)
model = YOLO('yolov8m.pt')  # medium model (even better accuracy)

# Start training - CORRECTED PATHS
results = model.train(
    data='./output/data.yaml',    # Use relative path to your data.yaml
    epochs=50,                    # Number of training epochs
    imgsz=800,                   # Image size
    batch=12,                    # Batch size (reduce if GPU memory issues)
    name='taco_detection',       # Experiment name
    patience=100,                 # Early stopping patience
    save=True,                   # Save model checkpoints
    plots=True,                  # Generate training plots
    project='./runs/detect',     # Local project directory
    device=0                     # Use GPU if available (or 'cpu' for CPU only)
)

print("🎉 Training complete!")
print(f"📁 Model saved in: ./runs/detect/taco_detection")

In [ ]:
# Load the trained model
model_path = './runs/detect/taco_detection/weights/best.pt'
model = YOLO(model_path)

print("📊 Evaluating model performance...")

# Run validation
metrics = model.val(data='./output/data.yaml')

print("\n" + "="*50)
print("🎯 MODEL PERFORMANCE SUMMARY")
print("="*50)
print(f"📈 mAP50 (IoU=0.5): {metrics.box.map50:.3f}")
print(f"📈 mAP50-95 (IoU=0.5:0.95): {metrics.box.map:.3f}")
print(f"🎯 Precision: {metrics.box.mp:.3f}")
print(f"🎯 Recall: {metrics.box.mr:.3f}")
print(f"⚡ Inference Speed: {metrics.speed['inference']:.1f}ms per image")
print("="*50)

# Performance interpretation
if metrics.box.map50 > 0.7:
    print("🌟 Excellent performance!")
elif metrics.box.map50 > 0.5:
    print("✅ Good performance!")
elif metrics.box.map50 > 0.3:
    print("⚠️  Moderate performance - consider more training")
else:
    print("❌ Low performance - check data quality or increase training")

In [ ]:
# Display training plots
from IPython.display import Image, display
import glob

print("📊 Training Results Visualization")

# Find result images - CORRECTED PATH
results_dir = './runs/detect/taco_detection'
plot_files = glob.glob(os.path.join(results_dir, '*.png'))

# Display key plots
key_plots = ['results.png', 'confusion_matrix.png', 'val_batch0_pred.png']

for plot_name in key_plots:
    plot_path = os.path.join(results_dir, plot_name)
    if os.path.exists(plot_path):
        print(f"\n📈 {plot_name.replace('_', ' ').title()}")
        display(Image(plot_path))
    else:
        print(f"⚠️  {plot_name} not found")

print(f"\n📁 All training results saved in: {results_dir}")

In [ ]:
# Test model on validation images
def test_on_validation_samples(model, output_dir, num_samples=6):
    """Test model on sample validation images"""

    val_images_dir = os.path.join(output_dir, 'images/val')
    results_dir = './inference_results'  # CORRECTED PATH
    os.makedirs(results_dir, exist_ok=True)

    # Get sample validation images
    val_images = [f for f in os.listdir(val_images_dir) if f.endswith('.jpg')]
    sample_images = random.sample(val_images, min(num_samples, len(val_images)))

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()

    print(f"🔍 Testing model on {len(sample_images)} validation images...")

    for i, img_file in enumerate(sample_images):
        img_path = os.path.join(val_images_dir, img_file)

        # Run inference
        results = model(img_path, conf=0.25)  # Confidence threshold

        # Get annotated image
        annotated_img = results[0].plot()
        annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

        # Count detections
        num_detections = len(results[0].boxes) if results[0].boxes is not None else 0

        # Display
        axes[i].imshow(annotated_img)
        axes[i].set_title(f'{img_file}\nDetections: {num_detections}')
        axes[i].axis('off')

        # Save annotated image
        output_path = os.path.join(results_dir, f'annotated_{img_file}')
        cv2.imwrite(output_path, cv2.cvtColor(annotated_img, cv2.COLOR_RGB2BGR))

        # Print detection details
        if num_detections > 0:
            print(f"📷 {img_file}: {num_detections} objects detected")
            for j, box in enumerate(results[0].boxes):
                class_id = int(box.cls)
                confidence = float(box.conf)
                class_name = model.names[class_id]
                print(f"   └─ {class_name}: {confidence:.2f}")
        else:
            print(f"📷 {img_file}: No objects detected")

    plt.tight_layout()
    plt.show()

    print(f"💾 Annotated images saved to: {results_dir}")

# Run validation test
test_on_validation_samples(model, './output')

In [ ]:
# Test model on custom images
def test_custom_image(model, image_path):
    """Test model on a specific image"""

    if not os.path.exists(image_path):
        print(f"❌ Image not found: {image_path}")
        return

    print(f"🔍 Testing model on: {os.path.basename(image_path)}")

    # Run inference
    results = model(image_path, conf=0.25)

    # Load and display original image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Get annotated image
    annotated_img = results[0].plot()
    annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

    # Display side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    ax1.imshow(image)
    ax1.set_title('Original Image')
    ax1.axis('off')

    ax2.imshow(annotated_img)
    ax2.set_title('YOLO Detection Results')
    ax2.axis('off')

    plt.tight_layout()
    plt.show()

    # Print detection details
    num_detections = len(results[0].boxes) if results[0].boxes is not None else 0
    print(f"🎯 Found {num_detections} objects:")

    if num_detections > 0:
        for i, box in enumerate(results[0].boxes):
            class_id = int(box.cls)
            confidence = float(box.conf)
            class_name = model.names[class_id]
            print(f"   {i+1}. {class_name} (confidence: {confidence:.2f})")

    return results

# Example usage (uncomment and modify path to test custom images):
# test_custom_image(model, './your_test_image.jpg')

print("💡 To test custom images, use: test_custom_image(model, 'path/to/image.jpg')")

In [ ]:
# Export model to different formats for deployment
print("📦 Exporting model for deployment...")

export_dir = './exported_models'  # CORRECTED PATH
os.makedirs(export_dir, exist_ok=True)

# Export to ONNX (cross-platform)
try:
    onnx_path = model.export(format='onnx', simplify=True)
    print(f"✅ ONNX model exported: {onnx_path}")
except Exception as e:
    print(f"❌ ONNX export failed: {e}")

# Export to TensorFlow Lite (mobile deployment)
try:
    tflite_path = model.export(format='tflite')
    print(f"✅ TensorFlow Lite model exported: {tflite_path}")
except Exception as e:
    print(f"❌ TFLite export failed: {e}")

# Export to TensorFlow SavedModel
try:
    saved_model_path = model.export(format='saved_model')
    print(f"✅ TensorFlow SavedModel exported: {saved_model_path}")
except Exception as e:
    print(f"❌ SavedModel export failed: {e}")

print("\n📋 Model Export Summary:")
print(f"🎯 Best PyTorch model: {model_path}")
print("📱 Use TFLite for mobile apps")
print("🌐 Use ONNX for web deployment")
print("☁️  Use SavedModel for TensorFlow Serving")

In [ ]:
# Create training summary report
import json
from datetime import datetime

# You'll need to load these variables if running this cell separately
# Uncomment and run if needed:
# json_path = './data/annotations.json'
# with open(json_path, 'r') as f:
#     data = json.load(f)
# images = data['images']
# annotations = data['annotations']
# categories = data['categories']

# Collect training information
training_summary = {
    "timestamp": datetime.now().isoformat(),
    "dataset": {
        "name": "TACO",
        "total_images": len(images),
        "total_annotations": len(annotations),
        "num_classes": len(categories),
        "train_images": len(train_ids),
        "val_images": len(val_ids)
    },
    "model": {
        "architecture": "YOLOv8n",
        "pretrained": True,
        "epochs": 50,
        "image_size": 640,
        "batch_size": 16
    },
    "performance": {
        "mAP50": float(metrics.box.map50),
        "mAP50_95": float(metrics.box.map),
        "precision": float(metrics.box.mp),
        "recall": float(metrics.box.mr),
        "inference_speed_ms": float(metrics.speed['inference'])
    },
    "files": {
        "model_path": model_path,
        "config_path": './output/data.yaml',
        "results_dir": results_dir
    }
}

# Save summary - CORRECTED PATH
summary_path = './training_summary.json'
with open(summary_path, 'w') as f:
    json.dump(training_summary, f, indent=2)

print("📊 TRAINING COMPLETED SUCCESSFULLY! 🎉")
print("="*60)
print("📋 SUMMARY:")
print(f"   🎯 Final mAP50: {metrics.box.map50:.3f}")
print(f"   ⚡ Inference Speed: {metrics.speed['inference']:.1f}ms")
print(f"   📁 Model Location: {model_path}")
print(f"   📄 Summary Report: {summary_path}")
print("="*60)
print("🚀 Your TACO object detection model is ready to use!")

# Display final class distribution
print(f"\n🏷️  DETECTED CLASSES ({len(categories)} total):")
for i, cat in enumerate(categories[:20]):  # Show first 20
    yolo_id = category_id_to_yolo_id[cat['id']]
    print(f"   {yolo_id:2d}: {cat['name']}")
if len(categories) > 20:
    print(f"   ... and {len(categories)-20} more classes")